In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [14]:
batch_size = 64
learning_rate = 1e-2
num_epochs = 3 # 训练次数
# 判断GPU是否可用
use_gpu = torch.cuda.is_available()

# # 下载训练集 MNIST 手写数字训练集
# # 数据是datasets类型的
# train_dataset = datasets.FashionMNIST(
#     root='./datasets', train=True, transform=transforms.ToTensor(), download=True)

# test_dataset = datasets.FashionMNIST(
#     root='./datasets', train=False, transform=transforms.ToTensor())
# #　将数据处理成 DataLoader
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # 选择打乱数据
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) # 选择不打乱数据

In [5]:
class neuralNetwork(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(neuralNetwork, self).__init__() # super() 函数是用于调用父类(超类)的一个方法
# Sequential()表示将一个有序的模块写在一起，也就相当于将神经网络的层按顺序放在一起，这样可以方便结构显示
        self.layer1 = nn.Sequential(
            nn.Linear(in_dim, n_hidden_1),
            nn.ReLU(True)) # 表示使用ReLU激活函数
        self.layer2 = nn.Sequential(
            nn.Linear(n_hidden_1, n_hidden_2),
            nn.ReLU(True))
        self.layer3 = nn.Sequential(
            nn.Linear(n_hidden_2, out_dim),
            nn.ReLU(True))

# 定义向前传播
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [6]:
model = neuralNetwork(28 * 28, 300, 100, 10)

In [7]:
criterion = nn.CrossEntropyLoss() # 定义损失函数类型，使用交叉熵
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # 定义优化器，使用随机梯度下降

In [13]:
# 开始模型训练
for epoch in range(num_epochs):
    print('*' * 10)
    print(f'epoch {epoch+1}')
    running_loss = 0.0 # 初始值
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1): # 枚举函数enumerate返回下标和值
        img, label = data
        img = img.view(img.size(0), -1) # 将图片展开为28*28
        # 使用GPU？
        if use_gpu:
            img = img.cuda()
            label = label.cuda()
        # 向前传播
        out = model(img) # 前向传播
        loss = criterion(out, label) # 计算loss
        running_loss += loss.item() # loss求和
        _, pred = torch.max(out, 1)
        running_acc += (pred == label).float().mean()
        # 向后传播
        optimizer.zero_grad() # 梯度归零
        loss.backward() # 后向传播
        optimizer.step() # 更新参数

        if i % 300 == 0:
            print(f'[{epoch+1}/{num_epochs}] Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
    print(f'Finish {epoch+1} epoch, Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
    for name, parms in model.named_parameters():
        if name in ['layer3.0.bias', 'layer3.0.bias']:
            # print('-->name:', name)
            print(name, '-->grad_requirs:', parms.requires_grad)
            # print('--weight', torch.mean(parms.data))
            print(name, '-->grad_value:', torch.mean(parms.grad))
            # if name == 'cnns.0.weight':
            # print(name, torch.mean(parms.data))

**********
epoch 1
[1/5] Loss: 0.794353, Acc: 0.765260
[1/5] Loss: 0.792761, Acc: 0.765260
[1/5] Loss: 0.795929, Acc: 0.765434
Finish 1 epoch, Loss: 0.796212, Acc: 0.765375
layer3.0.bias -->grad_requirs: True
layer3.0.bias -->grad_value: tensor(0.0013)
**********
epoch 2
[2/5] Loss: 0.796180, Acc: 0.762656
[2/5] Loss: 0.790753, Acc: 0.765938
[2/5] Loss: 0.789586, Acc: 0.767483
Finish 2 epoch, Loss: 0.790850, Acc: 0.767807
layer3.0.bias -->grad_requirs: True
layer3.0.bias -->grad_value: tensor(0.0036)
**********
epoch 3
[3/5] Loss: 0.789247, Acc: 0.766927
[3/5] Loss: 0.784501, Acc: 0.770312
[3/5] Loss: 0.784194, Acc: 0.770174
Finish 3 epoch, Loss: 0.785919, Acc: 0.769723
layer3.0.bias -->grad_requirs: True
layer3.0.bias -->grad_value: tensor(-0.0075)
**********
epoch 4
[4/5] Loss: 0.790819, Acc: 0.768385
[4/5] Loss: 0.784552, Acc: 0.770339
[4/5] Loss: 0.778672, Acc: 0.772899
Finish 4 epoch, Loss: 0.780221, Acc: 0.772455
layer3.0.bias -->grad_requirs: True
layer3.0.bias -->grad_value: te

In [16]:
for i, data in enumerate(train_loader, 1): # 枚举函数enumerate返回下标和值
        img, label = data
        print(img.requires_grad)
        print(img.grad)
        break

False
None
